In [1]:
# Import dependencies
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *

from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account
from web3 import Web3, middleware, Account

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
w3= Web3(Web3.HTTPProvider('http://localhost:8545'))
#enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
#set gas price strategy to built in "medium" algorithm (~est 5min per tx) 
# see https://web3py.readthedocs.io/en/stable/gas_price.html?highlight=gas
#see https://ethgastation.info/API for a more accuerate strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

from web3.middleware import geth_poa_middleware

In [2]:
!conda list bit

# packages in environment at /Users/matthewblair/opt/anaconda3:
#
# Name                    Version                   Build  Channel
bit                       0.7.2                    pypi_0    pypi
bitarray                  1.2.2                    pypi_0    pypi


In [3]:
# Load and set environment variables
load_dotenv()
mnemonic = os.getenv("mnemonic")
type(mnemonic)

str

In [4]:
# Create a function called `derive_wallets`

def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys



In [5]:
#Setting dictionary of coins to be used in the wallet

coins = {"eth", "btc-test", "btc"}
numderive = 3



In [6]:
#Loading mneomnic env

keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)



In [7]:
#Set Private Keys
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0xc92d285ef749d75004e6ab6cf1a9e16d85eb6bfc550a6117841995a3ec389e76"
"cNG3kBzDteXZJCPR1xWv7iMAfdA64GzpKJdY3NCP5NvjV2T6ksa2"


In [8]:
#print json

print(json.dumps(keys, indent=4, sort_keys=True))



{
    "btc": [
        {
            "address": "1K2ZS5TwvZ8EJADePgd645i9jQX56AtoUo",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "Kwhv6vFwxyNqjv9xWmo9kbuoFsz8bZurLZeXJCeeDjAgBWm7qeBQ",
            "pubkey": "0317725a5fff825bc47edd6ce08c8b1b3633fdc3515927565fe192dcae0d69645e",
            "pubkeyhash": "c5bdc0d4760c12018b7cd19317101717dbe9847c",
            "xprv": "xprvA3NL5uM7dCkHNPxrLUfgiZGsuzJR7gxvF9sD4PiNs87h3E3Cy9Vkw5YkQ3VtSwaLJSe8zStZ2w6CQLnzEtymvCAtfsoqi8mbA3fXpkWv7eS",
            "xpub": "xpub6GMgVQt1TaJaat3KSWCh5hDcU28uX9gmcNnorn7zRTefv2NMWgp1UssEFLYLLrJviWaA5XVFmL51yGVMX3W3Urz1u14aNBYKVJQ9GNnzogK"
        },
        {
            "address": "16QBWXmecywY7dRiaGhq7HjYvdeYkw2mTi",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "Ky9aYstD826WH9jNpqPYB4ddSH9s5QKpodFHdT8yw4zijoPUFc7X",
            "pubkey": "02017ac8ab4b1140678ad53ce9ee7151c7aefac3c0522c54d56bdd1cebeb2963bf",
            "pubkeyhash":

In [9]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mxcpHYVCcRUoeXT3ZboHwhv6AeUt1hPgD6>


In [10]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.


def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        value = w3.toWei(amount, "ether") # convert to wei
        gasEstimate = w3.eth.estimateGas(
            {
                "from": account.address, 
                "to": recipient, 
                "value": value
            }
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": value,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])



In [11]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [12]:
print(btc_acc)

<PrivateKeyTestnet: mxcpHYVCcRUoeXT3ZboHwhv6AeUt1hPgD6>


In [43]:
# create BTC transaction
create_trx(BTCTEST,btc_acc,"msVCUnAjTZ6wiKUMwR9Up7rPjs6bc1LmPZ", 0.0000000001)




'{"unspents":[{"amount":10457,"confirmations":10,"script":"76a914bb95f2dd1a82e2f83d77004bbb9d08f2d45870b888ac","txid":"a8d188f655aeb9dc804d4c7bbe62dd3abad0920f90e3c6d0420b0785e86dc164","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":100000,"confirmations":0,"script":"76a914bb95f2dd1a82e2f83d77004bbb9d08f2d45870b888ac","txid":"8676ed15a213bf2364c01a36dccdf5fedb425beb62ee03bf7b29c58d96f8db90","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":10000,"confirmations":0,"script":"76a914bb95f2dd1a82e2f83d77004bbb9d08f2d45870b888ac","txid":"7b153153ec0ff4ca2adadc186bd4223a511610977f318bfd6127401cb5279770","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["msVCUnAjTZ6wiKUMwR9Up7rPjs6bc1LmPZ",0],["mxcpHYVCcRUoeXT3ZboHwhv6AeUt1hPgD6",67213]]}'

In [44]:
#send BTC transaction
send_trx(BTCTEST,btc_acc,'msVCUnAjTZ6wiKUMwR9Up7rPjs6bc1LmPZ',0.00000001)
#mxcpHYVCcRUoeXT3ZboHwhv6AeUt1hPgD6

'010000000364c16de885070b42d0c6e3900f92d0ba3add62be7b4c4d80dcb9ae55f688d1a8010000006a4730440220363a2b10dab5c6a0c20c868dd3070326fbc4b372344c5ff9035ef0ef2b79a49402207f0ff27ea3402e317f806eba163de48f29d9000c7b84f354c0107a5fada539f001210253ceb56a6da2954b965e981ec065f1cd959d77255ed1109f2e23bac037e52725ffffffff90dbf8968dc5297bbf03ee62eb5b42dbfef5cddc361ac06423bf13a215ed7686010000006a47304402205ff7a91d4f6a5efd7370da7da5b165a2c00ec55141a249ec8100c249bf89f75502203d1d4a3c4c6694f1ff7bec48613897bd73454d88501ed7aded83fc150c018bd801210253ceb56a6da2954b965e981ec065f1cd959d77255ed1109f2e23bac037e52725ffffffff709727b51c402761fd8b317f971016513a22d46b18dcda2acaf40fec5331157b000000006a473044022069d710006ee057dd0f33a5526abfa5bfbcb52cf1ae17d98e6013c8e4e8a8412f022074413bfd4ae3f91088d5a90c33500ad8c6d244b2500ce1b81534f07d8046ca4001210253ceb56a6da2954b965e981ec065f1cd959d77255ed1109f2e23bac037e52725ffffffff0201000000000000001976a914834c84b9c171956e4b87cf2ff6bacbc545232c4e88ac8c060100000000001976a914bb95f2dd1a82e

In [14]:



#connecting to HTTP with address pk
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))



In [19]:
#create ETH transaction 
create_trx(ETH,eth_acc,"0x018A4c5B36d997aAc69F7Dbcb72b5f2dec703479",1000)

{'to': '0x018A4c5B36d997aAc69F7Dbcb72b5f2dec703479',
 'from': '0x07F597826b464919894F8aE6DA17C8796e01F250',
 'value': 1000000000000000000000,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [18]:
#check ETH balance
w3.eth.getBalance('0x07F597826b464919894F8aE6DA17C8796e01F250')

100000000000000000000

In [21]:
#send ETH transaction
send_trx(ETH, eth_acc,"0x018A4c5B36d997aAc69F7Dbcb72b5f2dec703479", 1)

0x6709b56148442077cd91e97e0d62a89b5c5eb40d5f981ed4234a8854f5a00202


'0x6709b56148442077cd91e97e0d62a89b5c5eb40d5f981ed4234a8854f5a00202'

In [22]:
#check ETH balance
w3.eth.getBalance('0x018A4c5B36d997aAc69F7Dbcb72b5f2dec703479')

101000000000000000000